## Часть 2.
---
Получим данные обо всех квартирах и соберем данные для последующего анализа.

In [20]:
# import packages

from datetime import datetime
import time
import pandas as pd
import pickle as pk


In [21]:
# open the output text file from '01_Scrape_All_Links_gh.ipynb'

with open('condo_links_all.txt') as f:
    condo_links_all = f.read().splitlines()
condo_links_all

['https://www.hipflat.co.th/en/projects/baan-lalin-in-the-park-rama-2-ekachai-wcnynk',
 'https://www.hipflat.co.th/en/projects/baan-mantika-fhiqfw',
 'https://www.hipflat.co.th/en/projects/baan-phet-wongwaen-vlkaic',
 'https://www.hipflat.co.th/en/projects/baan-phrapin-5-qwmptu',
 'https://www.hipflat.co.th/en/projects/baan-sunthorn-7-aefyds',
 'https://www.hipflat.co.th/en/projects/baan-thaweethong-2-kwexqe',
 'https://www.hipflat.co.th/en/projects/baan-nawarin-hgcvyi',
 'https://www.hipflat.co.th/en/projects/bangbon-garden-ville-jawexj',
 'https://www.hipflat.co.th/en/projects/centro-sukhumvit-113-uggfpo',
 'https://www.hipflat.co.th/en/projects/chuan-chuen-ekkachaibangbon-4-ophjay',
 'https://www.hipflat.co.th/en/projects/condo-u-at-phaholyothin-station-vtkjvc',
 'https://www.hipflat.co.th/en/projects/golden-prestige-ekachai-wongwaen-qzzlpi',
 'https://www.hipflat.co.th/en/projects/golden-prestige-ekkachaiwongwaen-jacinx',
 'https://www.hipflat.co.th/en/projects/keha-bangbon-igdbhp'

In [22]:

from bs4 import BeautifulSoup
import requests

def retrieve(link):
    page = requests.get(link)
    print(link)
    soup = BeautifulSoup(page.content, 'html.parser')
    
    graph_data=soup.find(id="graph1").get_text().strip()
    if graph_data != "Not enough data to build the graph":
        name=soup.find(itemprop="name").get_text().strip()
        district=soup.find(itemprop="streetAddress").get_text().strip().split(",")[-1].strip()
        latitude=str(soup.find(itemprop="latitude")).split("=")[1].split()[0]
        longitude=str(soup.find(itemprop="longitude")).split("=")[1].split()[0]
    
        description=str(soup.find(class_="property-description__content"))
        year_built=description.split(". Condominium")[0].split()[-1]
        buildings=description.split("building")[0].split()[-1]
        floors=description.split("floors")[0].split()[-1]
        units=description.split("units")[0].split()[-1]
        print(name,district,latitude,longitude,"\n",year_built,buildings,floors,units)
        
        neighborhood=[]
        for i in range(0,15):
            x=soup.find(class_="property-description__content").findAll('li')[i].get_text()
            neighborhood.append(x)
        
        shops=neighborhood[0:5]
        
            
        schools=neighborhood[5:10]
       
        
        restaurants=neighborhood[10:15]
       
        
        hospital=soup.find(class_="property-description__content").findAll('p')[-3].get_text()
        amenities=[]
        for i in range(0,12):
            if ('yes' in str(soup.find(class_="amenities").findAll('li')[i])):
                amenities.append(1)
            else:
                amenities.append(0)
        
        transportation=[]
        for i in range(0,5):
            tran_type=soup.findAll(class_="media neighborhood-destination")[i].find(class_="icon").i['class'][1]
            trans_name=soup.findAll(class_="media-heading")[i].get_text()
            trans_dist=soup.findAll(class_="media neighborhood-destination")[i].find('small').get_text()
            transportation.append((tran_type,trans_name,trans_dist))
            
  
        price_sqm=soup.find(class_="indicator__amount").find(class_="money").get_text().strip('฿').replace(',',"")
        change_last_q=soup.findAll(class_="indicator__amount")[1].get_text().replace('\n',"").strip()
        change_last_y=soup.findAll(class_="indicator__amount")[2].get_text().replace('\n',"").strip()
        rental_yield=soup.findAll(class_="indicator__amount")[3].get_text().replace('\n',"").strip()
        change_last_y_rental_price=soup.findAll(class_="indicator__amount")[4].get_text().replace('\n',"").strip()
       
        
        price_hist=soup.find(class_="row-fluid background-color-gray project__graph-container").find('script').get_text().split('\n')[3].strip().strip(',').replace('data: ',"")
        
        
        return (name,district,latitude,longitude,year_built,buildings,floors,units,\
           shops,schools,restaurants,hospital,amenities,transportation,\
           price_sqm,change_last_q,change_last_y,rental_yield,change_last_y_rental_price,price_hist)
    else:
        print("---------Not enough data to build the graph----------",'\n')

In [23]:


start_time = datetime.now()

condo_list=[]
i=0

for link in condo_links_all:
    try:
        condo_list.append(retrieve(link))
    except Exception: 
        pass
    print(i)
    time_elapsed = datetime.now() - start_time
    print('Time elapsed (hh:mm:ss.ms) {}'.format(time_elapsed))
    
    time.sleep(5)
    i=i+1

    
    if (i%5==0):
        condo_list = [c for c in condo_list if c is not None]
        df = pd.DataFrame(condo_list)
        with open('df.pkl', 'wb') as f:
            pk.dump(df, f)
        print('------------------------ dump @ i = ',i,len(condo_list))         
print("completed")

https://www.hipflat.co.th/en/projects/baan-lalin-in-the-park-rama-2-ekachai-wcnynk
0
Time elapsed (hh:mm:ss.ms) 0:00:00.208890
https://www.hipflat.co.th/en/projects/baan-mantika-fhiqfw
1
Time elapsed (hh:mm:ss.ms) 0:00:05.336255
https://www.hipflat.co.th/en/projects/baan-phet-wongwaen-vlkaic
2
Time elapsed (hh:mm:ss.ms) 0:00:10.469658
https://www.hipflat.co.th/en/projects/baan-phrapin-5-qwmptu
3
Time elapsed (hh:mm:ss.ms) 0:00:15.574728
https://www.hipflat.co.th/en/projects/baan-sunthorn-7-aefyds
4
Time elapsed (hh:mm:ss.ms) 0:00:20.670994
------------------------ dump @ i =  5 0
https://www.hipflat.co.th/en/projects/baan-thaweethong-2-kwexqe
5
Time elapsed (hh:mm:ss.ms) 0:00:25.811188
https://www.hipflat.co.th/en/projects/baan-nawarin-hgcvyi
6
Time elapsed (hh:mm:ss.ms) 0:00:30.926987
https://www.hipflat.co.th/en/projects/bangbon-garden-ville-jawexj
7
Time elapsed (hh:mm:ss.ms) 0:00:36.050955
https://www.hipflat.co.th/en/projects/centro-sukhumvit-113-uggfpo
8
Time elapsed (hh:mm:ss.ms

In [29]:
# Once complete, dump to pickle and save as 'df_completed.pkl'.
condo_list = [c for c in condo_list if c is not None]
df_completed = pd.DataFrame(condo_list)
with open('df_completed.pkl', 'wb') as f:
    pk.dump(df_completed, f)
    

In [ ]:
col_names= ['name','district','latitude','longitude','year_built','proj_area','nbr_buildings','nbr_floors','units',
            'shops','schools','restaurants','hospital','amenities','transportation',
            'price_sqm','change_last_q','change_last_y','rental_yield','change_last_y_rental_price','price_hist']
df_completed.to_csv("df_completed.csv" ,header=col_names,index=False,encoding='utf-8-sig')
df_dirty= pd.read_csv("df_completed.csv", sep=',',encoding='utf-8-sig')